# EXTRACT - Extração dos Dados

Importação dos dados através da API da Telecom X. Os dados estão em formato **JSON**.

Bibliotecas utilizadas no processo de Extração

In [131]:
import pandas as pd
import numpy as np
import requests
import re

Buscando os dados 

In [132]:
# Url da base de dados
data_url = 'https://raw.githubusercontent.com/ingridcristh/challenge2-data-science/refs/heads/main/TelecomX_Data.json'

response = requests.get(data_url)

if response.status_code == 200:
    json_file = response.json()
else:
    print(f"Bad response: status code: {response.status_code}")
    


In [133]:
# Criando o Dataframe
raw_data = pd.DataFrame(json_file)

# Primeira visualização
raw_data.head()

,customerID,Churn,customer,phone,internet,account
0,0002-ORFBO,No,"{'gender': 'Female', 'SeniorCitizen': 0, 'Part...","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'DSL', 'OnlineSecurity': '...","{'Contract': 'One year', 'PaperlessBilling': '..."
1,0003-MKNFE,No,"{'gender': 'Male', 'SeniorCitizen': 0, 'Partne...","{'PhoneService': 'Yes', 'MultipleLines': 'Yes'}","{'InternetService': 'DSL', 'OnlineSecurity': '...","{'Contract': 'Month-to-month', 'PaperlessBilli..."
2,0004-TLHLJ,Yes,"{'gender': 'Male', 'SeniorCitizen': 0, 'Partne...","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'Fiber optic', 'OnlineSecu...","{'Contract': 'Month-to-month', 'PaperlessBilli..."
3,0011-IGKFF,Yes,"{'gender': 'Male', 'SeniorCitizen': 1, 'Partne...","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'Fiber optic', 'OnlineSecu...","{'Contract': 'Month-to-month', 'PaperlessBilli..."
4,0013-EXCHZ,Yes,"{'gender': 'Female', 'SeniorCitizen': 1, 'Part...","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'Fiber optic', 'OnlineSecu...","{'Contract': 'Month-to-month', 'PaperlessBilli..."


O dataframe tem 4 colunas aninhadas que precisam ser tratadas para refletir as colunas do dicionário de dados.

In [134]:
# Criando uma lista com as colunas que precisam ser normalizadas
df = pd.json_normalize(json_file)
df.head()

,customerID,Churn,customer.gender,customer.SeniorCitizen,customer.Partner,customer.Dependents,customer.tenure,phone.PhoneService,phone.MultipleLines,internet.InternetService,...,internet.OnlineBackup,internet.DeviceProtection,internet.TechSupport,internet.StreamingTV,internet.StreamingMovies,account.Contract,account.PaperlessBilling,account.PaymentMethod,account.Charges.Monthly,account.Charges.Total
0,0002-ORFBO,No,Female,0,Yes,Yes,9,Yes,No,DSL,...,Yes,No,Yes,Yes,No,One year,Yes,Mailed check,65.6,593.3
1,0003-MKNFE,No,Male,0,No,No,9,Yes,Yes,DSL,...,No,No,No,No,Yes,Month-to-month,No,Mailed check,59.9,542.4
2,0004-TLHLJ,Yes,Male,0,No,No,4,Yes,No,Fiber optic,...,No,Yes,No,No,No,Month-to-month,Yes,Electronic check,73.9,280.85
3,0011-IGKFF,Yes,Male,1,Yes,No,13,Yes,No,Fiber optic,...,Yes,Yes,No,Yes,Yes,Month-to-month,Yes,Electronic check,98.0,1237.85
4,0013-EXCHZ,Yes,Female,1,Yes,No,3,Yes,No,Fiber optic,...,No,No,Yes,Yes,No,Month-to-month,Yes,Mailed check,83.9,267.4


Com os dados devidamente carregados em um Pandas DataFrame, agora vou fazer uma visão geral dos dados contidos nesse dataframe.
Utilizando o método **.info()** do Pandas, fica fácil ter uma apanhado geral do tipo de dados, nomes de colunas, número de linhas e colunas temos no dataframe.

In [135]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7267 entries, 0 to 7266
Data columns (total 21 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   customerID                 7267 non-null   object 
 1   Churn                      7267 non-null   object 
 2   customer.gender            7267 non-null   object 
 3   customer.SeniorCitizen     7267 non-null   int64  
 4   customer.Partner           7267 non-null   object 
 5   customer.Dependents        7267 non-null   object 
 6   customer.tenure            7267 non-null   int64  
 7   phone.PhoneService         7267 non-null   object 
 8   phone.MultipleLines        7267 non-null   object 
 9   internet.InternetService   7267 non-null   object 
 10  internet.OnlineSecurity    7267 non-null   object 
 11  internet.OnlineBackup      7267 non-null   object 
 12  internet.DeviceProtection  7267 non-null   object 
 13  internet.TechSupport       7267 non-null   objec

Utilizando o método .shape para ter de forma clara a quantidade de linhas e colunas.

In [136]:
rows, columns = df.shape
print(f'linhas: {rows}, colunas: {columns}')

linhas: 7267, colunas: 21


Criei um arquivo .py com o dicionário dos dados das colunas do dataset, isso irá me auxiliar a entender melhor o propósito de cada coluna do dataframe e dar o tratamento apropriado.

In [137]:
from dict_data import dict_data

for key, value in dict_data.items():
    print(f"{key}: {value}")

customerID: número de identificação único de cada cliente
Churn: se o cliente deixou ou não a empresa
gender: gênero (masculino e feminino)
SeniorCitizen:  informação sobre um cliente ter ou não idade igual ou maior que 65 anos 
Partner: se o cliente possui ou não um parceiro ou parceira
Dependents: se o cliente possui ou não dependentes
tenure:  meses de contrato do cliente
PhoneService: assinatura de serviço telefônico
MultipleLines: assisnatura de mais de uma linha de telefone
InternetService: assinatura de um provedor internet
OnlineSecurity: assinatura adicional de segurança online
OnlineBackup: assinatura adicional de backup online 
DeviceProtection: assinatura adicional de proteção no dispositivo
TechSupport: assinatura adicional de suporte técnico, menos tempo de espera
StreamingTV: assinatura de TV a cabo 
StreamingMovies: assinatura de streaming de filmes 
Contract: tipo de contrato
PaperlessBilling: se o cliente prefere receber online a fatura
PaymentMethod: forma de pagamen

## Transform - Transformação

Com os dados devidamente carregados e agora com as colunas identificadas através do dicionário de dados disponibilizado junto a API, iniciarei a limpeza dos dados 

#### Renomeando as colunas

O formato atual dos nomes de colunas no dataframe, está fora do padrão e irá dificultar a manipulação na hora de fazer os comandos, por isso com ajuda do dicionário de dados, estarei renomeando essas colunas para além de facilitar o processo de manipulação, padronizar os nomes das mesmas.

Abaixo, criei uma função que receberá o nome da coluna e retornará o nome em formato snakeCase.

In [138]:
#renomeando as colunas para facilitar

# Função para padronização dos nomes das colunas
def format_column_name(name: str) -> str:
    
    name = name.replace('.', '')  # remove pontos
    # insere _ antes de uma letra maiúscula que vem depois de uma letra minúscula ou número
    name = re.sub(r'(?<=[a-z0-9])(?=[A-Z])', '_', name)
    return name.lower()


In [139]:
# cria uma lista com o nome atual das colunas do dataframe
old_columns= df.columns.tolist() 

# Cria uma lista, utilizando o retorno da função map(), que recebe a função criada acima e a lista com o nome atual das colunas do dataframe e retorna uma nova lista com o nome padronizado das colunas.
new_columns= list(map(format_column_name, list(dict_data.keys())))

# Um dicionário utilizando a função zip(), onde a key é o nome antigo e o value, o nome novo
dict_columns = {key: value for key, value in zip(old_columns, new_columns)}
dict_columns

{'customerID': 'customer_id',
 'Churn': 'churn',
 'customer.gender': 'gender',
 'customer.SeniorCitizen': 'senior_citizen',
 'customer.Partner': 'partner',
 'customer.Dependents': 'dependents',
 'customer.tenure': 'tenure',
 'phone.PhoneService': 'phone_service',
 'phone.MultipleLines': 'multiple_lines',
 'internet.InternetService': 'internet_service',
 'internet.OnlineSecurity': 'online_security',
 'internet.OnlineBackup': 'online_backup',
 'internet.DeviceProtection': 'device_protection',
 'internet.TechSupport': 'tech_support',
 'internet.StreamingTV': 'streaming_tv',
 'internet.StreamingMovies': 'streaming_movies',
 'account.Contract': 'contract',
 'account.PaperlessBilling': 'paperless_billing',
 'account.PaymentMethod': 'payment_method',
 'account.Charges.Monthly': 'charges_monthly',
 'account.Charges.Total': 'charges_total'}

Com o dicionário de novos nomes pronto, é hora de renomear as colunas do dataframe.

In [140]:
df.rename(columns=dict_columns, inplace=True)

In [141]:
# Resultado
df.head()

,customer_id,churn,gender,senior_citizen,partner,dependents,tenure,phone_service,multiple_lines,internet_service,...,online_backup,device_protection,tech_support,streaming_tv,streaming_movies,contract,paperless_billing,payment_method,charges_monthly,charges_total
0,0002-ORFBO,No,Female,0,Yes,Yes,9,Yes,No,DSL,...,Yes,No,Yes,Yes,No,One year,Yes,Mailed check,65.6,593.3
1,0003-MKNFE,No,Male,0,No,No,9,Yes,Yes,DSL,...,No,No,No,No,Yes,Month-to-month,No,Mailed check,59.9,542.4
2,0004-TLHLJ,Yes,Male,0,No,No,4,Yes,No,Fiber optic,...,No,Yes,No,No,No,Month-to-month,Yes,Electronic check,73.9,280.85
3,0011-IGKFF,Yes,Male,1,Yes,No,13,Yes,No,Fiber optic,...,Yes,Yes,No,Yes,Yes,Month-to-month,Yes,Electronic check,98.0,1237.85
4,0013-EXCHZ,Yes,Female,1,Yes,No,3,Yes,No,Fiber optic,...,No,No,Yes,Yes,No,Month-to-month,Yes,Mailed check,83.9,267.4


Para me ajudar a entender melhor os dados, vou visualizar as colunas e a quantidade de dados distintos em cada uma, isso me ajudará a ver como as colunas estão distribuídas entre dados booleanos, categóricos e numéricos.

In [142]:
df.nunique()

customer_id          7267
churn                   3
gender                  2
senior_citizen          2
partner                 2
dependents              2
tenure                 73
phone_service           2
multiple_lines          3
internet_service        3
online_security         3
online_backup           3
device_protection       3
tech_support            3
streaming_tv            3
streaming_movies        3
contract                3
paperless_billing       2
payment_method          4
charges_monthly      1585
charges_total        6531
dtype: int64

Com essa visualização, é fácil identificar que das 21 colunas, 4 uma grande variedade de dados distintos e o restante, 17 colunas tem dados categóricos/ booleanos.

In [143]:
# Laço de repetição, para exibir as colunas com menos dados únicos(categóricas) e o tipo de dados que cada coluna tem.

for col in df.columns.tolist():
    if df[col].nunique() < 5:
        print(f'{col} => {df[col].unique()}')

churn => ['No' 'Yes' '']
gender => ['Female' 'Male']
senior_citizen => [0 1]
partner => ['Yes' 'No']
dependents => ['Yes' 'No']
phone_service => ['Yes' 'No']
multiple_lines => ['No' 'Yes' 'No phone service']
internet_service => ['DSL' 'Fiber optic' 'No']
online_security => ['No' 'Yes' 'No internet service']
online_backup => ['Yes' 'No' 'No internet service']
device_protection => ['No' 'Yes' 'No internet service']
tech_support => ['Yes' 'No' 'No internet service']
streaming_tv => ['Yes' 'No' 'No internet service']
streaming_movies => ['No' 'Yes' 'No internet service']
contract => ['One year' 'Month-to-month' 'Two year']
paperless_billing => ['Yes' 'No']
payment_method => ['Mailed check' 'Electronic check' 'Credit card (automatic)'
 'Bank transfer (automatic)']


Conhecendo o tipo de dado que está presente em cada coluna com poucos dados únicos. A maioria das colunas tem dados binários do tipo **(YES/NO)**, quando não acompanhado dos valores **(No phone service, No internet service, ou ' ')**, uma coluna com binário numérico **(0/1)** e 04 colunas categóricas **(gender, internet_service, contract, payment_method)**.

Para padronizar os dados, estarei modificando as colunas com dados de valores **(YES/NO)** para **(1/0)**, preservando os valores diferentes como strings vazias e 'No phone service', 'No internet service' como  **-1**. A preservação desses valores visa melhor análise pelos modelos de ML e para não tendenciar o resultado dos dados consistentes. Usar -1 é uma convenção comum para representar valores "especiais" ou categorias que sinalizam ausência de resposta ou dados não confiáveis. A vantagem é manter todos os dados como inteiros, o que facilita o uso em modelos que não lidam bem com NaN

In [144]:
#Lista com valores atuais das colunas que serão modificados
key_words = ['No', 'Yes', '', 'No phone service', 'No internet service']

# Lista com valores que substituirão os dados nas colunas
new_data = [0, 1, -1, -1, -1]

# Lista vazia que armazenará os nomes das colunas que terão os dados formatados.
data_to_change = []

for col in df.columns.tolist(): # Itera por todas as colunas
    if df[col].nunique() < 5:   # seleciona as com menos de 5 valores únicos.
        for  _ in df[col].unique().tolist(): # Itera por uma listas das colunas com menos de 5 valores únicos.
            if _ in key_words and col != 'internet_service':  # Verifica se algum dado da coluna está na lista key_words
                data_to_change.append(col) # Adiciona o nome da coluna a lista data_to_change.
                
print('++++++++++++++++++++++++++++++++')
print('Colunas para formatar os dados')
data_to_change = set(data_to_change)        # mantem apenas os valores únicos da lista
print(data_to_change )
dict_data_to_change = dict(zip(key_words, new_data))
print("Dicionário para mudança dos dados")
dict_data_to_change

++++++++++++++++++++++++++++++++
Colunas para formatar os dados
{'streaming_tv', 'multiple_lines', 'online_security', 'paperless_billing', 'partner', 'dependents', 'online_backup', 'phone_service', 'tech_support', 'churn', 'device_protection', 'streaming_movies'}
Dicionário para mudança dos dados


{'No': 0, 'Yes': 1, '': -1, 'No phone service': -1, 'No internet service': -1}

Com o dicionário criado, vou iterar sobre os dados atuais observando suas contagem para posterior comparação, a fim de evitar mudança nos dados.

In [145]:
for _ in data_to_change:
    print(f'{_}: {df[_].value_counts()}')
    print('==========================')

streaming_tv: streaming_tv
No                     2896
Yes                    2790
No internet service    1581
Name: count, dtype: int64
multiple_lines: multiple_lines
No                  3495
Yes                 3065
No phone service     707
Name: count, dtype: int64
online_security: online_security
No                     3608
Yes                    2078
No internet service    1581
Name: count, dtype: int64
paperless_billing: paperless_billing
Yes    4311
No     2956
Name: count, dtype: int64
partner: partner
No     3749
Yes    3518
Name: count, dtype: int64
dependents: dependents
No     5086
Yes    2181
Name: count, dtype: int64
online_backup: online_backup
No                     3182
Yes                    2504
No internet service    1581
Name: count, dtype: int64
phone_service: phone_service
Yes    6560
No      707
Name: count, dtype: int64
tech_support: tech_support
No                     3582
Yes                    2104
No internet service    1581
Name: count, dtype: int64
churn:

Com o dicionário dos dados que serão alterados pronto, utilizarei o método .replace para alterar os dados

In [146]:
df[list(data_to_change)] = df[list(data_to_change)].replace(dict_data_to_change)

/tmp/ipykernel_10805/3336162844.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[list(data_to_change)] = df[list(data_to_change)].replace(dict_data_to_change)


Verificando se os dados foram alterados sem alterar as quantidades.

In [147]:
for _ in data_to_change:
    print(f'{_}: {df[_].value_counts()}')
    print('==========================')

streaming_tv: streaming_tv
 0    2896
 1    2790
-1    1581
Name: count, dtype: int64
multiple_lines: multiple_lines
 0    3495
 1    3065
-1     707
Name: count, dtype: int64
online_security: online_security
 0    3608
 1    2078
-1    1581
Name: count, dtype: int64
paperless_billing: paperless_billing
1    4311
0    2956
Name: count, dtype: int64
partner: partner
0    3749
1    3518
Name: count, dtype: int64
dependents: dependents
0    5086
1    2181
Name: count, dtype: int64
online_backup: online_backup
 0    3182
 1    2504
-1    1581
Name: count, dtype: int64
phone_service: phone_service
1    6560
0     707
Name: count, dtype: int64
tech_support: tech_support
 0    3582
 1    2104
-1    1581
Name: count, dtype: int64
churn: churn
 0    5174
 1    1869
-1     224
Name: count, dtype: int64
device_protection: device_protection
 0    3195
 1    2491
-1    1581
Name: count, dtype: int64
streaming_movies: streaming_movies
 0    2870
 1    2816
-1    1581
Name: count, dtype: int64


In [148]:
for col in df.columns.tolist():
    if df[col].nunique() < 50:
        print(f'{col} => {df[col].unique()}')

churn => [ 0  1 -1]
gender => ['Female' 'Male']
senior_citizen => [0 1]
partner => [1 0]
dependents => [1 0]
phone_service => [1 0]
multiple_lines => [ 0  1 -1]
internet_service => ['DSL' 'Fiber optic' 'No']
online_security => [ 0  1 -1]
online_backup => [ 1  0 -1]
device_protection => [ 0  1 -1]
tech_support => [ 1  0 -1]
streaming_tv => [ 1  0 -1]
streaming_movies => [ 0  1 -1]
contract => ['One year' 'Month-to-month' 'Two year']
paperless_billing => [1 0]
payment_method => ['Mailed check' 'Electronic check' 'Credit card (automatic)'
 'Bank transfer (automatic)']


In [149]:
df.head()

,customer_id,churn,gender,senior_citizen,partner,dependents,tenure,phone_service,multiple_lines,internet_service,...,online_backup,device_protection,tech_support,streaming_tv,streaming_movies,contract,paperless_billing,payment_method,charges_monthly,charges_total
0,0002-ORFBO,0,Female,0,1,1,9,1,0,DSL,...,1,0,1,1,0,One year,1,Mailed check,65.6,593.3
1,0003-MKNFE,0,Male,0,0,0,9,1,1,DSL,...,0,0,0,0,1,Month-to-month,0,Mailed check,59.9,542.4
2,0004-TLHLJ,1,Male,0,0,0,4,1,0,Fiber optic,...,0,1,0,0,0,Month-to-month,1,Electronic check,73.9,280.85
3,0011-IGKFF,1,Male,1,1,0,13,1,0,Fiber optic,...,1,1,0,1,1,Month-to-month,1,Electronic check,98.0,1237.85
4,0013-EXCHZ,1,Female,1,1,0,3,1,0,Fiber optic,...,0,0,1,1,0,Month-to-month,1,Mailed check,83.9,267.4


In [150]:
df.dtypes

customer_id           object
churn                  int64
gender                object
senior_citizen         int64
partner                int64
dependents             int64
tenure                 int64
phone_service          int64
multiple_lines         int64
internet_service      object
online_security        int64
online_backup          int64
device_protection      int64
tech_support           int64
streaming_tv           int64
streaming_movies       int64
contract              object
paperless_billing      int64
payment_method        object
charges_monthly      float64
charges_total         object
dtype: object

Visualizando as últimas colunas que ainda estão como object

In [151]:
last_type_objects = df.select_dtypes(include=['object']).columns.tolist()
last_type_objects

['customer_id',
 'gender',
 'internet_service',
 'contract',
 'payment_method',
 'charges_total']

In [152]:
df[last_type_objects].head()

,customer_id,gender,internet_service,contract,payment_method,charges_total
0,0002-ORFBO,Female,DSL,One year,Mailed check,593.3
1,0003-MKNFE,Male,DSL,Month-to-month,Mailed check,542.4
2,0004-TLHLJ,Male,Fiber optic,Month-to-month,Electronic check,280.85
3,0011-IGKFF,Male,Fiber optic,Month-to-month,Electronic check,1237.85
4,0013-EXCHZ,Female,Fiber optic,Month-to-month,Mailed check,267.4


Das coslunas do tipo **Object** que restaram, apenas customer_id e charges_total não são do tipo **category**

In [154]:
last_type_objects = last_type_objects[1:5]

In [155]:
last_type_objects

['gender', 'internet_service', 'contract', 'payment_method']

In [156]:
df[last_type_objects].head()

,gender,internet_service,contract,payment_method
0,Female,DSL,One year,Mailed check
1,Male,DSL,Month-to-month,Mailed check
2,Male,Fiber optic,Month-to-month,Electronic check
3,Male,Fiber optic,Month-to-month,Electronic check
4,Female,Fiber optic,Month-to-month,Mailed check


Normalizando os dados e definindo o tipo como **category**

In [157]:
df[last_type_objects] = df[last_type_objects].apply(lambda x: x.str.lower()).astype('category')

In [158]:
df.dtypes

customer_id            object
churn                   int64
gender               category
senior_citizen          int64
partner                 int64
dependents              int64
tenure                  int64
phone_service           int64
multiple_lines          int64
internet_service     category
online_security         int64
online_backup           int64
device_protection       int64
tech_support            int64
streaming_tv            int64
streaming_movies        int64
contract             category
paperless_billing       int64
payment_method       category
charges_monthly       float64
charges_total          object
dtype: object

In [159]:
df.sample(5)

,customer_id,churn,gender,senior_citizen,partner,dependents,tenure,phone_service,multiple_lines,internet_service,...,online_backup,device_protection,tech_support,streaming_tv,streaming_movies,contract,paperless_billing,payment_method,charges_monthly,charges_total
5035,6890-PFRQX,0,male,0,0,0,18,1,1,dsl,...,0,1,0,1,0,month-to-month,0,mailed check,69.50,1199.4
1164,1658-TJVOA,0,female,1,0,0,6,1,0,fiber optic,...,0,0,0,0,0,month-to-month,1,electronic check,74.10,450.9
862,1219-NNDDO,0,female,0,0,0,9,1,1,fiber optic,...,0,0,0,0,0,month-to-month,1,electronic check,76.85,663.55
2407,3363-DTIVD,0,male,0,1,1,71,1,1,fiber optic,...,0,1,0,1,1,two year,0,electronic check,105.55,7405.5
1470,2108-GLPQB,0,male,0,1,0,25,1,1,dsl,...,0,1,0,0,0,month-to-month,0,credit card (automatic),55.55,1405.3


Terminado a limpeza/Transformação das colunas com poucos dados únicos, olhando agora para as colunas restantes e o tipo de dados contidos nelas.


In [160]:
for col in df.columns.tolist():
    if df[col].nunique() > 5:
        print(f'{col} => {df[col].unique()}')

customer_id => ['0002-ORFBO' '0003-MKNFE' '0004-TLHLJ' ... '9992-UJOEL' '9993-LHIEB'
 '9995-HOTOH']
tenure => [ 9  4 13  3 71 63  7 65 54 72  5 56 34  1 45 50 23 55 26 69 11 37 49 66
 67 20 43 59 12 27  2 25 29 14 35 64 39 40  6 30 70 57 58 16 32 33 10 21
 61 15 44 22 24 19 47 62 46 52  8 60 48 28 41 53 68 51 31 36 17 18 38 42
  0]
charges_monthly => [65.6  59.9  73.9  ... 91.75 68.8  67.85]
charges_total => ['593.3' '542.4' '280.85' ... '742.9' '4627.65' '3707.6']


O tipo da coluna **charges_total** está como object e seus dados aparentam ser do tipo string embora sejam números, possívelmente existem strings vazias ou numeros com caracteres especiais.

In [161]:
empty_strings= df.apply(lambda x: x.astype(str).str.strip() == '').sum()
empty_strings[empty_strings > 0]

charges_total    11
dtype: int64

In [162]:
df[df['charges_total'] == ' ']

,customer_id,churn,gender,senior_citizen,partner,dependents,tenure,phone_service,multiple_lines,internet_service,...,online_backup,device_protection,tech_support,streaming_tv,streaming_movies,contract,paperless_billing,payment_method,charges_monthly,charges_total
975,1371-DWPAZ,0,female,0,1,1,0,0,-1,dsl,...,1,1,1,1,0,two year,0,credit card (automatic),56.05,
1775,2520-SGTTA,0,female,0,1,1,0,1,0,no,...,-1,-1,-1,-1,-1,two year,0,mailed check,20.00,
1955,2775-SEFEE,0,male,0,0,1,0,1,1,dsl,...,1,0,1,0,0,two year,1,bank transfer (automatic),61.90,
2075,2923-ARZLG,0,male,0,1,1,0,1,0,no,...,-1,-1,-1,-1,-1,one year,1,mailed check,19.70,
2232,3115-CZMZD,0,male,0,0,1,0,1,0,no,...,-1,-1,-1,-1,-1,two year,0,mailed check,20.25,
2308,3213-VVOLG,0,male,0,1,1,0,1,1,no,...,-1,-1,-1,-1,-1,two year,0,mailed check,25.35,
2930,4075-WKNIU,0,female,0,1,1,0,1,1,dsl,...,1,1,1,1,0,two year,0,mailed check,73.35,
3134,4367-NUYAO,0,male,0,1,1,0,1,1,no,...,-1,-1,-1,-1,-1,two year,0,mailed check,25.75,
3203,4472-LVYGI,0,female,0,1,1,0,0,-1,dsl,...,0,1,1,1,0,two year,1,bank transfer (automatic),52.55,
4169,5709-LVOEQ,0,female,0,1,1,0,1,0,dsl,...,1,1,0,1,1,two year,0,mailed check,80.85,


Temos 11 linhas com strings vazias na coluna **charges_total**

Olhando para o dataset, pude observar que a coluna **charges_total** é o produto da multiplicação da coluna **ternure** pela coluna **charge_monthly** com uma pequena diferença que pode ser alguma taxa de serviço. 

Uma coisa em comum é que em todas as linhas de **charges_tota** com string vazia, a coluna **tenure** tem valor igual a 0(zero). Ou seja, pode ser um cliente novo, com contrato ainda não faturado, pois também não deixaram a empresa, como pode ser observado através da coluna **churn**.

In [163]:
sum(df['tenure'] == 0)

11

In [164]:
df[df['charges_total'] == ' '][['churn','tenure','charges_total']]

,churn,tenure,charges_total
975,0,0,
1775,0,0,
1955,0,0,
2075,0,0,
2232,0,0,
2308,0,0,
2930,0,0,
3134,0,0,
3203,0,0,
4169,0,0,


Os dados faltantes serão subistituídos pelo valor 0.

In [165]:
# Lista com os indíces em que aparece a (' ')
idx_empty_strings = df[df['charges_total'] == ' '].index.tolist()

# Substituindo por 0(zero).
df.loc[idx_empty_strings, 'charges_total'] = 0

# alterando o tipo da coluna para float
df['charges_total'] = df['charges_total'].astype(np.float64)

In [166]:
df.dtypes

customer_id            object
churn                   int64
gender               category
senior_citizen          int64
partner                 int64
dependents              int64
tenure                  int64
phone_service           int64
multiple_lines          int64
internet_service     category
online_security         int64
online_backup           int64
device_protection       int64
tech_support            int64
streaming_tv            int64
streaming_movies        int64
contract             category
paperless_billing       int64
payment_method       category
charges_monthly       float64
charges_total         float64
dtype: object

In [98]:
df.sample(10)

,customer_id,churn,gender,senior_citizen,partner,dependents,tenure,phone_service,multiple_lines,internet_service,...,online_backup,device_protection,tech_support,streaming_tv,streaming_movies,contract,paperless_billing,payment_method,charges_monthly,charges_total
4283,5889-JTMUL,1,female,1,1,0,50,1,1,fiber optic,...,1,1,0,1,0,month-to-month,1,electronic check,95.05,4888.70
2968,4118-CEVPF,0,female,1,0,0,35,1,1,fiber optic,...,1,1,1,1,1,one year,1,bank transfer (automatic),110.80,3836.30
6309,8677-HDZEE,0,female,0,0,0,56,1,1,fiber optic,...,1,0,1,1,1,month-to-month,1,bank transfer (automatic),105.45,5916.95
2721,3768-NLUBH,1,male,1,1,0,57,1,1,fiber optic,...,0,1,0,1,1,month-to-month,0,electronic check,100.60,5746.15
3040,4229-CZMLL,1,male,0,0,0,6,1,1,fiber optic,...,0,0,0,0,0,month-to-month,1,electronic check,74.90,469.80
351,0510-EXSMQ,-1,female,0,0,0,9,1,0,fiber optic,...,0,0,0,0,0,month-to-month,0,electronic check,69.05,651.50
4492,6161-UUUTA,1,male,1,0,0,3,1,0,fiber optic,...,0,0,0,1,0,month-to-month,1,electronic check,79.90,260.90
6506,8943-URTMR,1,female,0,0,0,2,1,0,fiber optic,...,0,0,0,1,0,month-to-month,1,mailed check,79.65,152.70
2487,3470-OBUET,0,female,0,1,1,67,1,1,dsl,...,1,0,1,1,0,two year,0,credit card (automatic),74.00,4868.40
6542,8992-CEUEN,0,female,0,0,0,1,1,0,no,...,-1,-1,-1,-1,-1,month-to-month,0,electronic check,18.85,18.85


### Enriquecendo o dataset

Como parte do desafio, foi sugerido a criação da coluna contas_diarias, utilizando o faturamento mensal para calcular o valor diário, proporcionando uma visão mais detalhada do comportamento dos clientes ao longo do tempo.

In [167]:
df['charges_daily'] = df['charges_monthly'] / 30

In [168]:
df.sample(5)

,customer_id,churn,gender,senior_citizen,partner,dependents,tenure,phone_service,multiple_lines,internet_service,...,device_protection,tech_support,streaming_tv,streaming_movies,contract,paperless_billing,payment_method,charges_monthly,charges_total,charges_daily
3523,4854-CIDCF,0,female,1,0,0,3,1,0,fiber optic,...,1,0,0,0,month-to-month,0,electronic check,73.85,196.40,2.461667
3409,4718-WXBGI,0,male,0,1,1,72,1,1,dsl,...,1,1,1,1,two year,1,credit card (automatic),91.95,6614.90,3.065000
2901,4018-PPNDW,0,female,0,1,1,61,1,1,no,...,-1,-1,-1,-1,two year,1,mailed check,24.10,1551.60,0.803333
6252,8606-CIQUL,0,male,1,1,1,72,1,1,fiber optic,...,1,1,1,1,two year,1,credit card (automatic),115.75,8399.15,3.858333
1966,2792-LSHWX,0,female,0,0,0,1,1,0,fiber optic,...,0,1,0,0,month-to-month,1,mailed check,83.40,83.40,2.780000


### Alterando a ordem das colunas

In [169]:
current_columns = df.columns.tolist()
current_columns[-1], current_columns[-3] = current_columns[-3], current_columns[-1]
current_columns[-1], current_columns[-2] = current_columns[-2], current_columns[-1] 

In [170]:
df = df[current_columns]

In [171]:
df.sample(5)

,customer_id,churn,gender,senior_citizen,partner,dependents,tenure,phone_service,multiple_lines,internet_service,...,device_protection,tech_support,streaming_tv,streaming_movies,contract,paperless_billing,payment_method,charges_daily,charges_monthly,charges_total
148,0237-YFUTL,0,female,0,1,0,50,1,1,fiber optic,...,0,1,1,1,month-to-month,1,bank transfer (automatic),3.655000,109.65,5405.80
5851,8000-REIQB,1,female,1,0,0,1,1,0,fiber optic,...,0,0,0,0,month-to-month,1,electronic check,2.331667,69.95,69.95
1695,2402-TAIRZ,0,female,0,0,0,37,1,0,fiber optic,...,1,0,0,0,one year,0,electronic check,2.668333,80.05,3019.10
4926,6728-WYQBC,0,male,0,0,0,1,1,0,no,...,-1,-1,-1,-1,month-to-month,0,bank transfer (automatic),0.698333,20.95,20.95
1675,2373-NTKOD,0,male,0,0,0,8,1,0,dsl,...,0,0,1,1,month-to-month,1,electronic check,2.208333,66.25,546.45


# Load - Salvando o DataSet 
Após a limpeza e alterações dos dados, é hora de salvar o dataset,

Com a conclusão do ETL - Extract, Transform, agora só falta o L de load. Decidí criar um banco de dados local para salvar o dados já limpos, facilitando o desempenho e consultas.

In [173]:
from sqlalchemy import create_engine, MetaData, Table, inspect
engine = create_engine('sqlite:///clened_telecomX_db/teleconX_db.db')
# Salvando no banco de dados
df.to_sql('teleconX_origin', engine, index=False, if_exists='replace')

7267

In [174]:
print(inspect(engine).get_table_names())

['teleconX_BR', 'teleconX_origin']


### Localizando o dataset para português brasileiro

Como uma atividade proposta, vou traduzir as colunas e os dados que podem ser traduzidos sem comprometer a integridade dos dados

In [175]:
ptbr_columns = ['id_cliente', 'contrato_cancelado', 'sexo', 'igual_maior_65', 'tem_parceiro','tem_dependente', 'meses_contrato', 'servicos_telefonicos', 'multiplas_linhas', 'servicos_internet', 'seguranca_online','backup_online','seguro_protecao','suporte_tecnico','tem_tv_cabo', 'tem_streaming_filmes', 'tipo_contrato','fatura_online','forma_pagamento','custo_diario','valor_mensal','valor_total']

In [176]:
dict_ptbr = dict(zip(df.columns.tolist(), ptbr_columns))
dict_ptbr

{'customer_id': 'id_cliente',
 'churn': 'contrato_cancelado',
 'gender': 'sexo',
 'senior_citizen': 'igual_maior_65',
 'partner': 'tem_parceiro',
 'dependents': 'tem_dependente',
 'tenure': 'meses_contrato',
 'phone_service': 'servicos_telefonicos',
 'multiple_lines': 'multiplas_linhas',
 'internet_service': 'servicos_internet',
 'online_security': 'seguranca_online',
 'online_backup': 'backup_online',
 'device_protection': 'seguro_protecao',
 'tech_support': 'suporte_tecnico',
 'streaming_tv': 'tem_tv_cabo',
 'streaming_movies': 'tem_streaming_filmes',
 'contract': 'tipo_contrato',
 'paperless_billing': 'fatura_online',
 'payment_method': 'forma_pagamento',
 'charges_daily': 'custo_diario',
 'charges_monthly': 'valor_mensal',
 'charges_total': 'valor_total'}

fazendo uma cópia do dataframe para renomear colunas em ptbr

In [177]:
df_br = df.copy()

In [178]:
df_br.rename(columns=dict_ptbr, inplace=True)
df_br.sample(5)

,id_cliente,contrato_cancelado,sexo,igual_maior_65,tem_parceiro,tem_dependente,meses_contrato,servicos_telefonicos,multiplas_linhas,servicos_internet,...,seguro_protecao,suporte_tecnico,tem_tv_cabo,tem_streaming_filmes,tipo_contrato,fatura_online,forma_pagamento,custo_diario,valor_mensal,valor_total
1255,1796-JANOW,0,female,0,0,0,2,1,0,no,...,-1,-1,-1,-1,month-to-month,0,mailed check,0.665000,19.95,38.15
2553,3566-HJGPK,0,male,0,0,0,1,1,0,dsl,...,0,0,0,0,month-to-month,1,mailed check,1.518333,45.55,45.55
3760,5168-MSWXT,0,male,0,1,1,8,1,0,fiber optic,...,1,0,1,1,month-to-month,1,electronic check,3.158333,94.75,759.55
5298,7240-FQLHE,1,female,1,1,0,18,1,1,fiber optic,...,0,1,1,1,month-to-month,0,bank transfer (automatic),3.325000,99.75,1836.25
2661,3705-RHRFR,0,female,0,1,0,69,1,1,fiber optic,...,0,0,1,1,two year,0,bank transfer (automatic),3.305000,99.15,6875.35


traduzindo dados

In [179]:
to_translate = ['sexo', 'servicos_internet', 'tipo_contrato', 'forma_pagamento']

In [180]:
dict_to_translate = {
    'male': 'masculino', 
    'female': 'feminino',
    'dsl': 'dsl-cabo', 
    'fiber optic': 'fibra optica', 
    'no' : 'sem serviço',
    'month-to-month': 'mensal', 
    'two year': 'dois anos', 
    'one year': 'anual',
    'electronic check': 'cheque eletrônico', 
    'mailed check': 'cheque via correios', 
    'bank transfer (automatic)': 'tranferência bancária automática', 
    'credit card (automatic)': 'cartão de crédito automático'
    }

In [181]:
for col in df_br.columns.tolist():
    df_br[col] = df_br[col].replace(dict_to_translate)

/tmp/ipykernel_10805/864760780.py:2: FutureWarning: The behavior of Series.replace (and DataFrame.replace) with CategoricalDtype is deprecated. In a future version, replace will only be used for cases that preserve the categories. To change the categories, use ser.cat.rename_categories instead.
  df_br[col] = df_br[col].replace(dict_to_translate)


In [182]:
for col in df_br.columns.tolist():
    print(df_br[col].value_counts())

id_cliente
0002-ORFBO    1
6614-VBEGU    1
6637-KYRCV    1
6635-MYYYZ    1
6635-CPNUN    1
             ..
3374-TTZTK    1
3374-PZLXD    1
3374-LXDEV    1
3373-YZZYM    1
9995-HOTOH    1
Name: count, Length: 7267, dtype: int64
contrato_cancelado
 0    5174
 1    1869
-1     224
Name: count, dtype: int64
sexo
masculino    3675
feminino     3592
Name: count, dtype: int64
igual_maior_65
0    6085
1    1182
Name: count, dtype: int64
tem_parceiro
0    3749
1    3518
Name: count, dtype: int64
tem_dependente
0    5086
1    2181
Name: count, dtype: int64
meses_contrato
1     634
72    369
2     246
3     207
4     185
     ... 
38     60
39     59
44     54
36     50
0      11
Name: count, Length: 73, dtype: int64
servicos_telefonicos
1    6560
0     707
Name: count, dtype: int64
multiplas_linhas
 0    3495
 1    3065
-1     707
Name: count, dtype: int64
servicos_internet
fibra optica    3198
dsl-cabo        2488
sem serviço     1581
Name: count, dtype: int64
seguranca_online
 0    3608
 1    

In [184]:
df_br.sample(5)

,id_cliente,contrato_cancelado,sexo,igual_maior_65,tem_parceiro,tem_dependente,meses_contrato,servicos_telefonicos,multiplas_linhas,servicos_internet,...,seguro_protecao,suporte_tecnico,tem_tv_cabo,tem_streaming_filmes,tipo_contrato,fatura_online,forma_pagamento,custo_diario,valor_mensal,valor_total
3073,4285-GYRQC,1,feminino,0,1,0,7,1,0,fibra optica,...,1,0,1,0,mensal,1,cheque eletrônico,2.875000,86.25,587.10
4910,6712-OAWRH,1,feminino,1,0,0,26,1,0,fibra optica,...,0,0,1,1,mensal,1,cheque eletrônico,3.041667,91.25,2351.80
3059,4257-GAESD,0,feminino,0,0,0,6,0,-1,dsl-cabo,...,0,0,1,1,mensal,0,cartão de crédito automático,1.500000,45.00,298.70
3086,4299-SIMNS,0,masculino,0,0,0,9,1,1,fibra optica,...,0,0,0,0,mensal,1,cheque eletrônico,2.705000,81.15,784.45
1617,2294-SALNE,0,masculino,0,1,1,23,1,0,fibra optica,...,1,1,0,0,anual,0,cheque via correios,2.893333,86.80,1940.80


In [185]:
for col in df_br.columns.tolist():
    if df_br[col].nunique() < 50:
        print(f'{col} => {df_br[col].unique()}')

contrato_cancelado => [ 0  1 -1]
sexo => ['feminino', 'masculino']
Categories (2, object): ['feminino', 'masculino']
igual_maior_65 => [0 1]
tem_parceiro => [1 0]
tem_dependente => [1 0]
servicos_telefonicos => [1 0]
multiplas_linhas => [ 0  1 -1]
servicos_internet => ['dsl-cabo', 'fibra optica', 'sem serviço']
Categories (3, object): ['dsl-cabo', 'fibra optica', 'sem serviço']
seguranca_online => [ 0  1 -1]
backup_online => [ 1  0 -1]
seguro_protecao => [ 0  1 -1]
suporte_tecnico => [ 1  0 -1]
tem_tv_cabo => [ 1  0 -1]
tem_streaming_filmes => [ 0  1 -1]
tipo_contrato => ['anual', 'mensal', 'dois anos']
Categories (3, object): ['mensal', 'anual', 'dois anos']
fatura_online => [1 0]
forma_pagamento => ['cheque via correios', 'cheque eletrônico', 'cartão de crédito automático', 'tranferência bancária automática']
Categories (4, object): ['tranferência bancária automática', 'cartão de crédito automático', 'cheque eletrônico', 'cheque via correios']


In [186]:
df_br.dtypes

id_cliente                object
contrato_cancelado         int64
sexo                    category
igual_maior_65             int64
tem_parceiro               int64
tem_dependente             int64
meses_contrato             int64
servicos_telefonicos       int64
multiplas_linhas           int64
servicos_internet       category
seguranca_online           int64
backup_online              int64
seguro_protecao            int64
suporte_tecnico            int64
tem_tv_cabo                int64
tem_streaming_filmes       int64
tipo_contrato           category
fatura_online              int64
forma_pagamento         category
custo_diario             float64
valor_mensal             float64
valor_total              float64
dtype: object

### Salvando o dataset em PT-BR no banco de dados.

In [187]:
df_br.to_sql('teleconX_BR', engine, index=False, if_exists='replace')
print(inspect(engine).get_table_names())

['teleconX_BR', 'teleconX_origin']
